In [1]:
'''BARCLAYS OPTION TRADING STRATEGY'''
#This strat aims to exploit the gap between implied option vol and historical vol (volatility risk premium)
#Retail traders in the past years have created huge liquidity for short-dated calls on large cap stocks
#We build a strat around this that basically buys undervalued calls and sells overvalued ones
#The expiration date we select is going to be 2 weeks, as this is where most liquidiy is concentrated

'BARCLAYS OPTION TRADING STRATEGY'

In [20]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import dotenv
import os
import pandas as pd
import warnings
import datetime
import requests
from pymongo import MongoClient

warnings.filterwarnings('ignore')

dotenv.load_dotenv()
ALPHA_API = os.getenv("ALPHA_VANTAGE")
MONGO_URI = os.getenv("MONGO_URI")

client = MongoClient(MONGO_URI)
db = client['Barclays-Options']
collection = db['options-data']


In [4]:
'''Taking data from spy stocks to build the strat around'''
#We parse the wikipedia page for spy data

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
tickers = sp500['Symbol'].unique().tolist()

data = yf.download(tickers, period='1y')
data = data.stack()
data.index.names = ['date', 'ticker']
data.columns = data.columns.str.lower()

[*********************100%***********************]  503 of 503 completed


In [5]:
'''Well now filter by dollar volume to only opearte with the stocks which fit our liquidity constraints'''
#We will compute the mean dollar volume from the past year to filter liquid stocks
#Further, we selecct large cap stocks, as retail trader liquidity is more likely concentrated here

latest_date = data.index.get_level_values('date').max() #we use this below for option vol

data['dollarvol'] = (data['close'] * data['volume']) / 1e6
data = data.groupby('ticker').mean() #finding mean dollar vol over past year

data = data.sort_values(by='dollarvol', ascending = False)
data = data.head(20) #top 20 liquid stocks

In [6]:
'''Lets try to build a options volume column in the dataframe'''
#I tweaked with the weeks value and found the most liquidity to be in 2 weeks expriration date options, so we go with that
#This is an incredibly slow process, as yfinance doesn't give option volume directly and we have to find a workaround
#We now filter top 10 stocks with highest option volume
#Liquidity is either most concentrated in 1 week expiry options or 2 week expiry options

'''BUG => Too slow when trying to first filter by option volume, so did that later'''

data['option_volume'] = None
data['sector'] = None

latest_tickers = data.index.get_level_values('ticker')

count = 1

for tckr in latest_tickers:
    #expiration_date = (latest_date + timedelta(weeks=2)).strftime('%Y-%m-%d') #or tckr.options[1]

    tk = yf.Ticker(tckr)
    expiration_date = tk.options[1]
    opt_chain = tk.option_chain(expiration_date)

    option_volume = opt_chain.calls['volume'].sum() + opt_chain.puts['volume'].sum()

    data.loc[tckr, 'option_volume'] = option_volume
    data.loc[tckr, 'sector'] = tk.info.get('sector', 'NA')

    print("Ticker ", count, " done")
    count += 1

Ticker  1  done
Ticker  2  done
Ticker  3  done
Ticker  4  done
Ticker  5  done
Ticker  6  done
Ticker  7  done
Ticker  8  done
Ticker  9  done
Ticker  10  done
Ticker  11  done
Ticker  12  done
Ticker  13  done
Ticker  14  done
Ticker  15  done
Ticker  16  done
Ticker  17  done
Ticker  18  done
Ticker  19  done
Ticker  20  done


In [7]:
'''Checking how different max_dollar_volume and max_option_volume tickers are'''

data = data.sort_values(by='dollarvol', ascending = False)
max_dollar_vol = data.head(10)

data = data.sort_values(by='option_volume', ascending = False)
max_option_vol = data.head(10)

max_dvol= set(max_dollar_vol.index.to_list())
max_ovol = set(max_option_vol.index.to_list())

max_dvol.difference(max_ovol)
max_ovol.difference(max_dvol)

data = max_option_vol

In [8]:
'''Now, we will create a hashmap that maps each sector to its exchange traded fund(if it exists)'''
#We will need this later for the stat arb strategy
#There is no easy way to do this, so well select all popular etfs, view the sector of the top holding of each and create a map

etfs = [
    'XLK',  # Technology Select Sector SPDR Fund
    'XLF',  # Financial Select Sector SPDR Fund
    'XLV',  # Health Care Select Sector SPDR Fund
    'XLY',  # Consumer Discretionary Select Sector SPDR Fund
    'XLP',  # Consumer Staples Select Sector SPDR Fund
    'XLE',  # Energy Select Sector SPDR Fund
    'XLB',  # Materials Select Sector SPDR Fund
    'XLI',  # Industrial Select Sector SPDR Fund
    'XLU',  # Utilities Select Sector SPDR Fund
    'XLRE', # Real Estate Select Sector SPDR Fund
    'XLC',  # Communication Services Select Sector SPDR Fund
    'EEM',  # iShares MSCI Emerging Markets ETF
    'SPY',  # SPDR S&P 500 ETF Trust
    'VTI',  # Vanguard Total Stock Market ETF
    'VTV'   # Vanguard Value ETF
]

sector_to_etf = {}

for etf in etfs:
    etf_data = yf.Ticker(etf).funds_data.top_holdings
    etf_ticker = etf_data.index[0] #first holding
    sector = yf.Ticker(etf_ticker).info.get('sector', 'NA')
    sector_to_etf[sector] = etf

In [9]:
'''NOW COMES THE FUN QUANT PART'''
#We need a VolScore metric that can identify the spread of volatility risk premium
#Barclays doesn't reveal what they used to calculate volScore
#An educated guess (based on traditional pairs trading) is that we compare implied vol to historical vol AND sector vol
#Underlying assumption is that vol deviations are temporary (mean reversion)
#The formula I agree on is:

#            IV - (w1 * HV  +  w2 * SV)
# VolScore = --------------------------
#                    sigma_res

#w1 and w2 are weights that we'll determine with RollingOLS or Kalman Filter
#Weights should be time-varying preferably
#There is no need to weight IV too as the rationale is that IV already embeds the risk premium
#This comes from training GARCH models research - Univ of North Carolina research

# IV = w0 + w1 * HV + w2 * SV + epsilon_t
# epsilon_t is our error term, IV - IV_hat or residual
# IV is actual implied volatility and IV_hat is what our model estimates it to be
# The error term in the reason we normalize as we can't say if a given residue is significant or just market noise

#sigma_res is the standard deviation of residuals
#historical deviation of IV from weighted avg of HV and SV
#This normalization lets us use VolScore like a z-score, which lets us better gauge tradding opps

'''CONSIDERATIONS'''
#Use HFT data for HV as it helps capture intraday price movements that daily data can miss
#this reduces noise in data as shown by past research - Barndorff-Nielsen and Shephard
#Might have to do more research on how many years of data is best suited for pairs trading, curr is 1 yr

'CONSIDERATIONS'

In [10]:
'''Downloading etf data'''
#We will need this to compute the historical volatility of the sector
#We'll compute the daily volatilites for every day, mean them and find the annualized avg vol


req_etfs = [sector_to_etf[s] for s in data.sector.unique()]
tickers = " ".join(req_etfs)

etf_data = yf.download(tickers, period="2y")
etf_data = etf_data.stack()
etf_data.index.names = ['date', 'ticker']
etf_data.columns = etf_data.columns.str.lower()

[*********************100%***********************]  3 of 3 completed


In [11]:
'''Computing historical volatility'''

def calculate_hv(df, col):
    df = df.sort_index(level='date')
    
    returns = df['close'] / df['close'].shift(1)
    df['log_returns'] = np.log(returns)
    rolling_vol = df['log_returns'].rolling(window=252).std()
    df[col] = rolling_vol * np.sqrt(252)  # annual vol
    
    return df

In [12]:
# etff = yf.download('XLY', period='1mo')
# etff = etff.sort_values(by = "Date", ascending=False)
# etff = etff.head(2)
# print(etff.Close.XLY[-1], etff.Close.XLY[0])
# np.log(etff.Close.XLY[0] / etff.Close.XLY[-1])

In [13]:
'''Now we download the data for the 10 stocks we selected'''
#These contain all the actual option contracts we will be trading
#The dataframe consists of a IV section, which we'll use in the model we created

tickers = " ".join(data.index.tolist())

options_data = yf.download(tickers, period="2y")
options_data = options_data.stack()
options_data.index.names = ['date', 'ticker']
options_data.columns = options_data.columns.str.lower()

[*********************100%***********************]  10 of 10 completed


In [14]:
etf_data = etf_data.groupby("ticker").apply(calculate_hv, 'HV').dropna()
options_data = options_data.groupby("ticker").apply(calculate_hv, 'RV').dropna()

In [15]:
'''Run this script only once, otherwise necessary cols will be deleted'''

etf_data = etf_data.reset_index(level=2, drop=True) #removing duplicate ticker index
options_data = options_data.reset_index(level=2, drop=True)

In [16]:
'''Modifying the options_data dataframe to include the HV for the corresponding ticker-etf pair'''
#BUG: for loop was very slow here, so i ended up using previous dataframes mapping

options_data['sector'] = options_data.index.get_level_values('ticker').map(data['sector'])
options_data['ETF'] = options_data['sector'].map(sector_to_etf)

func = lambda row: etf_data.loc[(row['ETF'], row.name[1]), 'HV'] if (row['ETF'], row.name[1]) in etf_data.index else None

options_data['SV'] = options_data.apply(func, axis=1)

options_data.drop(columns=['ETF', 'sector'], axis=1, inplace=True)
options_data = options_data.dropna()

In [ ]:
'''Downloading options data for past two years to compute VIX'''
#Im using alphavantage API as this was the best service that I could find for free
#However alphavantage doesn't have a good way to directly fetch data as a dataframe
#It gives responses in json, which Im manually converting into a pd Df
#Because alphavantage rate limits (25 per day), ill store data to a MongoDB cluster first

two_yrs = (datetime.datetime.today() - datetime.timedelta(weeks=104)).strftime('%Y-%m-%d')

for ticker in data.index.get_level_values('ticker').tolist():
    url = 'https://www.alphavantage.co/query?' \
    'function=HISTORICAL_OPTIONS&symbol='+ticker+'&apikey='+ALPHA_API+'&date='+two_yrs

    r = requests.get(url)

    push_data = r.json()
    push_data['_id'] = ticker
    collection.insert_one(push_data)
    print("Data pushed to cluster for " + ticker)

In [90]:
'''Script to convert json data to Pandas dataframe'''
'''Script to query for data of a specific ticker from our Mongo cluster'''

def json_to_df(json_data): #json_data should be r.json()['data']
    ticker_options = pd.DataFrame()
    for day_data in json_data:
        ticker_options = pd.concat([ticker_options, pd.DataFrame([day_data])], ignore_index=True)

    return ticker_options

def get_data(ticker):
    mongo_data = collection.find_one({"_id": ticker})
    return "Data not found for " + ticker if mongo_data is None else json_to_df(mongo_data['data'])

In [ ]:
'''Now we need to compute the implied volatitlies of each ticker'''
#We run into a problem where each option has option contracts with differing volatilities
#A simple average wont cut it as options differ in ATM and ITM status, and in moneyness
#A weighted average is a good place to start
#We can also build our own VIX like model which I'm going to go with as its mathematically more robust



['NVDA',
 'TSLA',
 'AAPL',
 'AMZN',
 'PLTR',
 'META',
 'GOOGL',
 'AVGO',
 'SMCI',
 'AMD']

In [143]:
'''Building the linear regression model'''
#Now we build the linear regression model for VolScore based on the formula above
#we use HV from etf_data and RV from options_data for the weights

from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

def compute_volscore(df, window):
    df = df.sort_index(level = 'date').copy()
    